In [1]:
import pandas as pd
from pathlib import Path

## remove invalid letters from static keypoints

In [10]:
import pandas as pd
from pathlib import Path

keypoint_classifier_label = Path('../data/dataset/keypoint_classifier_label.csv')
final_static_csv = Path('../data/dataset/ngt_static_keypoint.csv')
additional_data_path = Path('../data/dataset/additional_data/')
additional_data_glob = list(additional_data_path.glob('*.csv'))

INVALID_LETTERS = list('BCFGKLMNOPQRST')
invalid_indices = [ord(letter) - ord('A') for letter in INVALID_LETTERS]
print(INVALID_LETTERS)
print(invalid_indices)



['B', 'C', 'F', 'G', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T']
[1, 2, 5, 6, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [11]:
# Step 1: Clean existing data
df = pd.read_csv(final_static_csv, header=None)

print(f"Original shape: {df.shape}")
print(f"Label distribution before:\n{df[0].value_counts().sort_index()}")

df_cleaned = df[~df[0].isin(invalid_indices)]

print(f"\nCleaned shape: {df_cleaned.shape}")
print(f"Label distribution after:\n{df_cleaned[0].value_counts().sort_index()}")



Original shape: (16976, 43)
Label distribution before:
0
0     1898
3     2275
4     3840
8     3592
21    1227
22    1965
24    2179
Name: count, dtype: int64

Cleaned shape: (16976, 43)
Label distribution after:
0
0     1898
3     2275
4     3840
8     3592
21    1227
22    1965
24    2179
Name: count, dtype: int64


In [12]:
# Step 2: Add new data from additional files
expected_cols = df_cleaned.shape[1]
print(f"\n{'='*50}")
print(f"Step 2: Adding new data")
print(f"Expected columns: {expected_cols}")
print(f"Found {len(additional_data_glob)} CSV files to process")

rows_added = 0
rows_skipped = 0

for csv_path in additional_data_glob:
    print(f"\nProcessing {csv_path.name}...")
    df_new = pd.read_csv(csv_path, header=None)
    
    if df_new.shape[1] != expected_cols:
        print(f"  WARNING: Column mismatch! Expected {expected_cols}, got {df_new.shape[1]}")
        print(f"  Skipping all {df_new.shape[0]} rows")
        rows_skipped += df_new.shape[0]
        continue
    
    df_cleaned = pd.concat([df_cleaned, df_new], ignore_index=True)
    rows_added += df_new.shape[0]
    print(f"  Added {df_new.shape[0]} rows")

# Step 3: Save final result
df_cleaned.to_csv(final_static_csv, header=False, index=False)

print(f"\n{'='*50}")
print(f"Final Summary:")
print(f"  Rows added: {rows_added}")
print(f"  Rows skipped: {rows_skipped}")
print(f"  Final shape: {df_cleaned.shape}")
print(f"  Final label distribution:\n{df_cleaned[0].value_counts().sort_index()}")
print(f"\nSaved to {final_static_csv}")


Step 2: Adding new data
Expected columns: 43
Found 1 CSV files to process

Processing aaron_0123.csv...
  Added 23668 rows

Final Summary:
  Rows added: 23668
  Rows skipped: 0
  Final shape: (40644, 43)
  Final label distribution:
0
0     1898
1     1592
2     1610
3     2275
4     3840
5     1666
6     1722
8     3592
10    1898
11    1668
12    1664
13    1580
14    1675
15    1682
16    1707
17    1818
18    1710
19    1676
21    1227
22    1965
24    2179
Name: count, dtype: int64

Saved to ../data/dataset/ngt_static_keypoint.csv
